In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from numpy import newaxis
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

In [ ]:
from pyfem.topo import Interval
from pyfem.poly import gll_points
from pyfem.sem import SEMhat
from pyfem.poly import eval_lagrange_d0 as eval_phi1d

In [ ]:
order = 4
semh = SEMhat(order)

N = 10
n_dofs = (order+1)*N

In [ ]:
L = 1.0

vertices = np.linspace(0, L, N+1)
etv      = np.zeros((N, 2), dtype=np.int)
etv[:,0] = np.arange(N)
etv[:,1] = np.arange(N)+1


topo  = Interval()
xq = topo.ref_to_phys(vertices[etv], semh.xgll)
jacb_det = topo.calc_jacb(vertices[etv])[0]

etv[-1,-1] = etv[0,0]
jacb_det

In [ ]:
# Make elem to dof map
etd = np.arange(N*(order+1))
etd = etd.reshape((N, -1))

#dof_phys = np.unique(xq)[:-1]
dof_phys = xq.ravel()

# Make Q
cols = etd.ravel()
rows = np.arange(len(cols))
vals = np.ones(len(cols))
Q = sps.coo_matrix((vals,(rows,cols)))

In [ ]:
# Averaging operator
rows = etd[:,[0,-1]].ravel()
cols = etv.ravel()
vals = np.ones_like(cols)

FtoDof = sps.coo_matrix((vals, (rows, cols)),
                       shape=(n_dofs,N)).tocsr()
AVG = FtoDof.dot(FtoDof.T)/2.0

# Extract face DOFS
vals = np.ones(len(rows))
FDofs = sps.coo_matrix((vals, (rows, rows))).tocsr()
vals[::2] = -1
SDofs = sps.coo_matrix((vals, (rows, rows))).tocsr()

In [ ]:
# Build A and Bl
Cl = sps.kron(sps.eye(N), semh.Ch)
C  = Q.T.dot(Cl.dot(Q))
C  = C.tocsr()

# Bl = sps.kron(sps.eye(N), semh.Bh*jacb_det)
# B  = Q.T.dot(Bl.dot(Q))
# B  = B.tocsr()
# Binv_data = 1.0/B.data

In [ ]:
# Build full elemental mass matrix
x, w = topo.get_quadrature(order+1)
P = eval_phi1d(semh.xgll, x).T
G = sps.dia_matrix((w, 0), shape=(len(x), len(x)))
Bf = P.T.dot(G.dot(P))*jacb_det
Bfinv = np.linalg.inv(Bf)

Blinv = sps.kron(sps.eye(N), Bfinv)
Binv  = Q.T.dot(Blinv.dot(Q))
Binv  = Binv.tocsr()

In [ ]:
# Time stepping matrix
F = Binv.dot(-C+SDofs.dot(FDofs-AVG))

In [ ]:
# Problem setup
a  = 100.*(dof_phys-.5)**2
u0 = np.exp(-a)

dt = 0.001
nt = int(10.0/dt)
nt, dt, nt*dt

In [ ]:
# Integrate with AB3
ue = u0
u  = u0.copy()

ca = [[1.0, 0.0, 0.0],
      [1.5, -.5, 0.0],
      [23./12, -16./12, 5./12]]
ca = np.array(ca, dtype=np.double)
fa = np.zeros((3, n_dofs))

for k in range(nt):
        
    ic = np.min([k, 2])
    fa[0,:] = F.dot(u)
    f = ca[ic].dot(fa)
    fa[1:,:] = fa[:-1,:]
    
    u = u+dt*f
    
u_ab3 = u
np.max(np.abs(u-ue))

In [ ]:
# Integrate with RK4
ue = u0
u  = u0.copy()

for k in range(nt):
    
    k1 = F.dot(u)
    k2 = F.dot(u+(dt/2.0)*k1)
    k3 = F.dot(u+(dt/2.0)*k2)
    k4 = F.dot(u+dt*k3)
    
    u = u+(dt/6.0)*(k1+2*k2+2*k3+k4)
    
u_rk4 = u
np.max(np.abs(u-ue))

In [ ]:
plt.plot(dof_phys[:-1], u_ab3[:-1])
plt.plot(dof_phys[:-1], u_rk4[:-1])
plt.plot(dof_phys[:-1], u0[:-1], 'b--')
np.max(np.abs(u_rk4-u_ab3))